In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Si no tienes ipywidgets instalado:
# pip install ipywidgets
from ipywidgets import interact, FloatSlider

# --- Parámetros del problema ---
p2 = 1.0
I = 1.2

p1_old = 1.0   # precio original del bien 1
p1_new = 1.1   # precio aumentado del bien 1 (ligero aumento)

# Parámetros de la utilidad
beta1 = 1.0
beta21 = 2.0
beta22 = 0.5
beta3 = 1.0
gamma = 2.0  # fijado

# --- Función de utilidad u(x1,x2; theta) ---
def utility(x1, x2, theta):
    """
    u(x1,x2;theta) = (A^{1-gamma})/(1-gamma) + theta * (B^{1-gamma})/(1-gamma)
    con gamma=2, A = x1 + 2 x2 - 1, B = x1 + 0.5 x2 - 1
    => u = -1/A - theta/B
    Solo está definida cuando A>0 y B>0.
    """
    x1 = np.asarray(x1, dtype=float)
    x2 = np.asarray(x2, dtype=float)

    A = beta1 * x1 + beta21 * x2 - beta3
    B = beta1 * x1 + beta22 * x2 - beta3

    u = np.full_like(A, np.nan, dtype=float)
    mask = (A > 0) & (B > 0)
    u[mask] = -1.0 / A[mask] - theta / B[mask]
    return u

# --- Óptimo sobre la recta presupuestaria (búsqueda en una dimensión) ---
def opt_on_budget(theta, p1, ngrid=2000):
    """
    Dados theta y p1, maximiza u(x1, x2; theta) sujeto a:
    p1*x1 + p2*x2 = I, x1>=0, x2>=0.
    Usamos búsqueda en una malla de x1 y x2 viene de la recta presupuestaria.
    """
    x1_max = I / p1
    # Evitamos exactamente los extremos para no tener problemas numéricos
    x1_grid = np.linspace(1e-4, x1_max - 1e-4, ngrid)
    x2_grid = (I - p1 * x1_grid) / p2

    u_grid = utility(x1_grid, x2_grid, theta)

    # Si todo es NaN, no hay punto interior en el dominio
    if np.all(np.isnan(u_grid)):
        return np.nan, np.nan, np.nan

    idx = np.nanargmax(u_grid)
    return x1_grid[idx], x2_grid[idx], u_grid[idx]

# --- Gráfico para un valor de theta ---
def plot_for_theta(theta):
    fig, ax = plt.subplots(figsize=(7, 6))

    # Malla para curvas de indiferencia
    x1_vals = np.linspace(0.0, I / min(p1_old, p1_new), 200)
    x2_vals = np.linspace(0.0, I / p2, 200)
    X1, X2 = np.meshgrid(x1_vals, x2_vals)
    U = utility(X1, X2, theta)

    # Óptimos con precio original y con precio aumentado
    x1_o, x2_o, u_o = opt_on_budget(theta, p1_old)
    x1_n, x2_n, u_n = opt_on_budget(theta, p1_new)

    # Curvas de indiferencia que pasan por cada óptimo
    levels = []
    for u_level in [u_o, u_n]:
        if np.isfinite(u_level):
            levels.append(u_level)

    # Añadimos un nivel intermedio para ver mejor la forma de las curvas
    if np.isfinite(u_o) and np.isfinite(u_n):
        levels.append((u_o + u_n) / 2.0)

    levels = sorted(set(levels))

    if levels:
        cs = ax.contour(X1, X2, U, levels=levels, linewidths=1.0)
        ax.clabel(cs, inline=True, fontsize=8)

    # Dos rectas presupuestarias
    for p1, style, label in [
        (p1_old, "-", "Presupuesto precio original"),
        (p1_new, "--", "Presupuesto precio aumentado"),
    ]:
        x1_b = np.linspace(0, I / p1, 200)
        x2_b = (I - p1 * x1_b) / p2
        ax.plot(x1_b, x2_b, style, label=label)

    # Canasta óptima con precio original
    ax.scatter(x1_o, x2_o, s=50, zorder=3,
               label="Óptimo (precio original)")

    # Canasta óptima con precio aumentado
    ax.scatter(x1_n, x2_n, s=50, zorder=3,
               label="Óptimo (precio aumentado)")

    # Flecha que va de la canasta con precio original
    # a la canasta con precio aumentado (mismo theta)
    ax.annotate(
        "",
        xy=(x1_n, x2_n),
        xytext=(x1_o, x2_o),
        arrowprops=dict(arrowstyle="->", linewidth=2)
    )

    # Etiquetas, límites y formato
    ax.set_xlabel(r"$x_1$ (bien 1)")
    ax.set_ylabel(r"$x_2$ (bien 2)")
    ax.set_xlim(0, I / min(p1_old, p1_new))
    ax.set_ylim(0, I / p2)
    ax.set_title(rf"Curvas de indiferencia y dos presupuestos  ($\theta = {theta:.3f}$)")
    ax.legend(loc="upper right")
    ax.grid(True)
    plt.show()

# --- Slider interactivo para theta ---
theta_slider = FloatSlider(
    min=0.02,
    max=0.24,
    step=0.005,
    value=0.05,
    description=r'$\theta$'
)

interact(plot_for_theta, theta=theta_slider);


interactive(children=(FloatSlider(value=0.05, description='$\\theta$', max=0.24, min=0.02, step=0.005), Output…